# Information-based learning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from typing import List, Dict

In [ ]:
# How does probability work, e.g. probability per playing card
probability_distribution = [1 / 52 for _ in range(52)]   

shannon_entropy = -np.sum(probability_distribution * np.log2(probability_distribution))
print(f'Total entropy: {round(shannon_entropy, 3)} bits')

## 1. Util functions

### DataSet wrapper

In [ ]:
class Feature: 
    def __init__(self, D, f):
        self._D = D
        self._f = f
    
    def filter(self, l): 
        return Feature(self._D[self._D[self._f] == l], self._f)
    
    def IsHomogenous(self) -> bool:
        return self._D[self._f].nunique() == 1
    
    @property
    def Mode(self) -> str:
        return self._D[self._f].mode().iloc[0]
    
    @property
    def levels(self) -> List[str]:
        return self._D[self._f].unique()
    
    @property
    def rows(self) -> int:
        return self._D.shape[0]

class DataSet:
    def __init__(self, D: pd.DataFrame, t: str):
        self._D = D
        self._t = t

    def feature(self, feature) -> Feature: 
        return Feature(self._D, feature)
    
    def desc(self, feature) -> Feature: 
        return Feature(self._D, feature)
    
    def target(self) -> Feature: 
        return Feature(self._D, self._t)
    
    def filter(self, f, l): 
        return DataSet(self._D[self._D[f] == l], self._t)
    
    @property
    def rows(self) -> int:
        return self._D.shape[0]

    @property
    def df(self): 
        return self._D 

### Basic Entropy functions

In [ ]:
def P(l, f: Feature):
    return f.filter(l).rows / f.rows

def H(f: str, D: DataSet) -> float:
    entropy = lambda P: P * np.log2( P )
    
    t = D.feature(f)
    return -sum([ entropy( P(l, t) ) for l in t.levels])

def REM(f: str, desc_feature: str, D: DataSet) -> float: 
    d = D.feature(desc_feature)
    
    # sum ( weight of level * entropy of level) 
    return sum([ P(l, d) * H(f, D.filter(desc_feature, l)) for l in d.levels])

## 2. Formula's

In [ ]:
data = [
    ['Ace', 'Hearts'], ['Two', 'Hearts'], ['Three', 'Hearts'], ['Four', 'Hearts'], ['Five', 'Hearts'], ['Six', 'Hearts'], ['Seven', 'Hearts'], ['Eight', 'Hearts'], ['Nine', 'Hearts'], ['Ten', 'Hearts'], ['Jack', 'Hearts'], ['Queen', 'Hearts'], ['King', 'Hearts'],
    ['Ace', 'Diamonds'], ['Two', 'Diamonds'], ['Three', 'Diamonds'], ['Four', 'Diamonds'], ['Five', 'Diamonds'], ['Six', 'Diamonds'], ['Seven', 'Diamonds'], ['Eight', 'Diamonds'], ['Nine', 'Diamonds'], ['Ten', 'Diamonds'], ['Jack', 'Diamonds'], ['Queen', 'Diamonds'], ['King', 'Diamonds'],
    ['Ace', 'Clubs'], ['Two', 'Clubs'], ['Three', 'Clubs'], ['Four', 'Clubs'], ['Five', 'Clubs'], ['Six', 'Clubs'], ['Seven', 'Clubs'], ['Eight', 'Clubs'], ['Nine', 'Clubs'], ['Ten', 'Clubs'], ['Jack', 'Clubs'], ['Queen', 'Clubs'], ['King', 'Clubs'],
    ['Ace', 'Spades'], ['Two', 'Spades'], ['Three', 'Spades'], ['Four', 'Spades'], ['Five', 'Spades'], ['Six', 'Spades'], ['Seven', 'Spades'], ['Eight', 'Spades'], ['Nine', 'Spades'], ['Ten', 'Spades'], ['Jack', 'Spades'], ['Queen', 'Spades'], ['King', 'Spades']
]

playing_cards = pd.DataFrame(data, columns=['Value', 'Category'])

D = DataSet(playing_cards, 'Category')

In [ ]:
# Probability per Category
P('Clubs', D.feature('Category'))

In [ ]:
# Entropy based on a descriptive feature
H('Category', D)

## 3. ID3 Algorithm

#### 3.1 Entropy functions

In [ ]:
def P(l, f: Feature):
    return f.filter(l).rows / f.rows

def H(D: DataSet) -> float:
    entropy = lambda P: P * np.log2( P )
    t = D.target()
    return -sum([ entropy( P(l, t) ) for l in t.levels])

def REM(desc_feature: str, D: DataSet) -> float: 
    d = D.feature(desc_feature)
    
    # sum ( weight of level * entropy of level) 
    return sum([ P(l, d) * H(D.filter(desc_feature, l)) for l in d.levels])

#### 3.2 Tree datastructure

In [ ]:
class Node:
    def show(self, indent=0) -> str: 
        pass
    
    def predict(self, instance: pd.Series):
        pass

class Leaf(Node): 
    def __init__(self, feature):
        self.feature = feature

    def show(self, indent=0) -> str: 
        print(f"{indent*' '}- Leaf: {self.feature}")

    def predict(self, _): 
        return self.feature

class Composite(Node): 
    def __init__(self, feature):
        self.feature = feature
        self.children: Dict[Any, Node] = {}

    def add(self, option, node):
        self.children[option] = node

    def mode(self, option):
        self._mode = option

    def show(self, indent=0) -> str: 
        print(f"{indent*' '}Node: {self.feature}")
        for option in self.children:
            print(f"{indent*' '}- Option: {option}")
            self.children[option].show(indent+4)

    def predict(self, instance: pd.Series): 
        val = instance[self.feature]
        if(val in self.children):
            return self.children[val].predict(instance)
        else:
            return self.children[self._mode].predict(instance)
        

#### 3.3 Algorithm

In [ ]:
def ID3(d: List[str], D: DataSet) -> Node:
     
    if(D.rows < 1): 
        raise Exception("DataSet is empty.") 
    
    if(len(d) < 1): 
        return Leaf( D.target().Mode )
    
    if(D.target().IsHomogenous()):
        return Leaf( D.target().levels[0] )

    MAX_IG = pd.Series(
        [ H(D) - REM(f, D) for f in d], 
        index=d
    ).idxmax()

    decision_node = Composite(MAX_IG)
    
    modeDensity = 0
    for l in D.df[MAX_IG].unique():
        partition = D.filter(MAX_IG, l)

        child = D.target().Mode if partition.rows < 1 else ID3(list(filter(lambda l: l != MAX_IG, d)), partition)
        decision_node.add(l, child)

        pDensity = partition.rows / D.rows
        if(pDensity > modeDensity):
            decision_node.mode(l)
            modeDensity = pDensity

    return decision_node

## 4. Spam-ham example

In [ ]:
data = [
    [367,True,False,True,'SPAM'],
    [489,True,True,False,'SPAM'],
    [541,True,True,False,'SPAM'],
    [693,False,True,True,'HAM'],
    [782,False,False,False,'HAM'],
    [976,False,False,False,'HAM'],
] 
features = ['ID', 'Suspicious words', 'Unknown sender', 'Contains images', 'Class']
spam_emails = pd.DataFrame(data, columns=features)

spam_emails

In [ ]:
t: str = 'Class'
d: List[str] = features[1:4]
D: DataSet = DataSet(spam_emails, t='Class')

In [ ]:
# Information Gain
# IG = H(target_feature, D) - REM(descriptive_feature, D)
pd.Series(
    [ H(D) - REM(f, D) for f in d], 
    index=d
)

#### Algorithm

In [ ]:
tree = ID3(d, D)
tree.print()

In [ ]:
new_record = pd.Series({'Suspicious words': True, 'Unknown sender': False, 'Contains images': True})
tree.predict(new_record)

## 5. Real world dataset

### Setup 

In [ ]:

df = pd.read_csv('../datasets/drugs.decision-trees.csv', sep=',', names=["Age","Sex", "BP", "Cholesterol", "Na_to_K", "Drug"], header=0)
# df['AgeGroup'] = pd.cut(df['Age'], bins=range(10, 80, 5), right=False, labels=False) # age range is between 15 and 74

d = ["Age", "Sex", "BP", "Cholesterol"]
t = 'Drug'

df.head(5)


### Data check

In [ ]:
categorial_cols = ["Age", "AgeGroup", "Sex", "BP", "Cholesterol"]
headers = ["feature","count", "% miss.", "card.", "Mode", "Mode freq.", "Mode %", "2nd Mode", "2nd Mode freq.", "2nd Mode %"]
report = []
for feature in categorial_cols:
    report.append([
        feature,
        df[feature].size,
        df[feature].isnull().sum() / df[feature].size,
        df[feature].nunique(),
        df[feature].mode().values[0],
        df[feature].value_counts().max(),
        (df[feature].value_counts().max() / df[feature].size) * 100,
        df[feature].value_counts().index[1],
        df[feature].value_counts().iloc[1],
        (df[feature].value_counts().iloc[1] / df[feature].size) * 100,
    ])
pd.DataFrame(report, columns=headers)

In [ ]:
df['Drug'].value_counts().plot(kind='bar', title='Drug')

## 6. Real world dataset II

In [ ]:
features = ['buying price', 'maintenance cost', 'nr of doors', 'nr of persons', 'lug boot', 'safety', 'decision']
df = pd.read_csv('../datasets/car_evaluation.decision-trees.csv', sep=',', names=features, header=0)

d = features[:-1]
t = 'decision'

df.shape

## 7. Validation

In [ ]:
train, test = train_test_split(df, test_size=0.3) 

D: DataSet = DataSet(train, t)

tree = ID3(d, D)
test['Pred'] = df.apply(lambda x: tree.predict(x), axis=1)
test.head(5)

In [ ]:
train, test = train_test_split(df, test_size=0.1) 

D: DataSet = DataSet(train, t)

tree = ID3(d, D)

test['Pred'] = df.apply(lambda x: tree.predict(x), axis=1)
test.head(5)

In [ ]:
new_record = pd.Series({"AgeGroup": 47, "Sex": "M", "BP": "LOW", "Cholesterol": "HIGH"})
tree.predict(new_record)

In [ ]:
# Create a new DataFrame with unique 'Target' levels
accuracy_df = pd.DataFrame(test[t].unique(), columns=[t])

# Calculate the count of correct and incorrect predictions
accuracy_df['Correct'] = accuracy_df[t].apply(lambda x: ((test[t] == x) & (test[t] == x)).sum())
accuracy_df['Incorrect'] = accuracy_df[t].apply(lambda x: ((test[t] == x) & (test[t] != x)).sum())
accuracy_df['Accuracy'] = round(accuracy_df['Correct'] / (accuracy_df['Correct'] + accuracy_df['Incorrect']),3)

accuracy_df.loc[len(accuracy_df.index)] = ['Total', accuracy_df['Correct'].sum(), accuracy_df['Incorrect'].sum(), accuracy_df['Accuracy'].mean()] 

accuracy_df